In [ ]:
import geopandas as gpd
import sys
from pathlib import Path

# Set up path
%run ../bootstrap.py
setup_project_path()

from scripts.io_helpers import get_project_root

# Get project root
project_root = get_project_root()
sys.path.append(str(project_root))

from scripts import config

In [ ]:
# Load dataset
gdf = gpd.read_file(config.INTERIM_PATH, layer="Water_Flow_CO")

print(gdf.columns.tolist())
gdf.head()